In [1]:
!pip install sdv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.5/198.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.3/198.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 43.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.5 MB/s 

In [3]:
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import SingleTableMetadata
import pandas as pd
import random

#Fetch Data
url = "https://raw.githubusercontent.com/shrikant-temburwar/Loan-Prediction-Dataset/master/train.csv"
df = pd.read_csv(url)

df = df[['Gender', 'Education', 'Self_Employed', 'ApplicantIncome',
         'CoapplicantIncome', 'LoanAmount', 'Credit_History',
         'Property_Area', 'Loan_Status']]

#Clean
df = df.dropna()
df['Credit_History'] = df['Credit_History'].astype(str)

#Train CTGAN
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(df)
model = CTGANSynthesizer(metadata)
model.fit(df)

#Generate 1000 Synthetic Records
synthetic_data = model.sample(1000)

reasons = [
    "I need to renovate my house immediately.",
    "Planning to expand my small business operations.",
    "Urgent medical expenses covered by loan.",
    "Buying a new car for daily commute.",
    "Consolidating high-interest debts.",
    "Investment in a new startup opportunity."
]

qualifiers = [
    "I am very confident in repayment.",
    "I might struggle slightly initially.",
    "Business is booming right now.",
    "Facing some financial stress currently."
]

def generate_text(row):
    reason = random.choice(reasons)
    if row['ApplicantIncome'] > 10000:
        qualifier = "I am very confident in repayment."
    else:
        qualifier = random.choice(qualifiers)
    return f"{reason} {qualifier}"

synthetic_data['Application_Note'] = synthetic_data.apply(generate_text, axis=1)

#Save
print(synthetic_data.head())
synthetic_data.to_csv('synthetic_financial_data_enhanced.csv', index=False)

/usr/local/lib/python3.11/dist-packages/sdv/single_table/base.py:168: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
/usr/local/lib/python3.11/dist-packages/sdv/single_table/base.py:134: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


   Gender     Education Self_Employed  ApplicantIncome  CoapplicantIncome  \
0    Male      Graduate            No             1969           0.000000   
1    Male  Not Graduate            No             1544           0.000000   
2    Male      Graduate            No              388         345.682079   
3    Male  Not Graduate           Yes              150         145.880326   
4  Female      Graduate           Yes             1691          32.047091   

   LoanAmount Credit_History Property_Area Loan_Status  \
0       247.0            0.0         Rural           N   
1       268.0            1.0     Semiurban           Y   
2       121.0            1.0     Semiurban           Y   
3       216.0            1.0     Semiurban           Y   
4       103.0            1.0         Rural           N   

                                    Application_Note  
0  Urgent medical expenses covered by loan. Facin...  
1  Urgent medical expenses covered by loan. Busin...  
2  Buying a new car for